# Use Recurrent Neural Network to generate text
<br>

## Li Linwei

<br>
## Introduction<br>
<font size=3>
In this Implementation, I build an Recurrent Neural Network to train the text from an novel and use the trained model to generate a text with similar style using few chracters. <br>Since every character inside of a word has certain connected from each other, so I use LSTM network as hidden layer, they can save the state from previous character and based on the information from previous to predict what should be the next chracter. Some of the graphics are selected from Andrej Karpathy's post on RNNs.
</font>
<br>
<br>

## Framework<br>
<font size=3>
The following figure depicts the overall framework, if we have the word <font size=4>'hello'</font>, we will enter the characters sequentially: <br>
<font size=4>'h => e => l => l', </font><font size=3>the targets are the inputs shifted over one character, so if the prediction is right, the result should be <br></font><font size=4>'e => l => l => o'</font> <font size=3>sequentially.</font><br><br>
<img src="assets/charseq.jpeg" width="500">
<br>

## Input Layer
<br>
<font size=3>
Since a text has thousands of word, every word has many characters, so we need to divide the text into batch, like the pictur below, and feed every batch to inputs at one time, so the </font><font size=4.5>Input layer</font> <font size=3>should be:
<br><br>
<img src="assets/sequence_batching@1x.png" width=500px>
<br><br><br>
The details of the network can be illustrated as the figure below:</font><br><br>
<img src="assets/charRNN.png" width=500px>
<br><br>

## Hidden Layer ( LSTM )<br>
<font size=3>I set the number of hidden layer as 2, that's a recommended number experimentally. Every LSTM layer has 512 hidden units, and I also use Dropout in the spatial output of every LSTM layer, the Dropout probablity is 0.5.</font><br><br>


## Output Layer<br>
<font size=3>The </font> <font size=4.5>output layer</font> <font size=4.5> uses softmax as the activation function, which can provided us a sequence of probability distribution, the network can determine the most probable character according to this distribution and predict the next character, so the number of output should be the number of characters which ever existed in total text, include some characters like blank or punctuation. <br></font>

## Loss<br>
<font size=3>The </font> <font size=4.5>loss</font> <font size=3>of output is calculated by cross-entropy of prediction sequence and target sequence. We also add a gradients_clip as threshold in order to prevent gradients explode during gradients descent, That is, if a gradient is larger than that threshold, we set it to the threshold.
</font>


## Optimizer<br>
<font size=3> And as for the optimizer we use </font> <font size=4.5> Adam Optimizer</font>,<font size=3> which has been recommended in many place.</font>
<br>
## Train
<font size=3>
The Hyperparameters are set as follow:
</font>
```python
batch_size = 100        # Sequences per batch
num_steps = 100         # Number of sequence steps per batch
lstm_size = 512         # Size of hidden layers in LSTMs
num_layers = 2          # Number of LSTM layers
learning_rate = 0.001   # Learning rate
keep_prob = 0.5         # Dropout keep probability
```
<br>
<font size=3>
The </font><font size=4.5>last episode</font><font size=3> of train results are shown below:
```python
Epoch: 20/20...  Training Step: 3951...  Training loss: 1.0423...  2.5772 sec/batch
Epoch: 20/20...  Training Step: 3952...  Training loss: 1.0606...  2.5905 sec/batch
Epoch: 20/20...  Training Step: 3953...  Training loss: 1.0346...  2.6036 sec/batch
Epoch: 20/20...  Training Step: 3954...  Training loss: 1.0432...  2.6674 sec/batch
Epoch: 20/20...  Training Step: 3955...  Training loss: 1.0548...  2.6038 sec/batch
Epoch: 20/20...  Training Step: 3956...  Training loss: 1.0414...  2.5892 sec/batch
Epoch: 20/20...  Training Step: 3957...  Training loss: 1.0342...  2.5783 sec/batch
Epoch: 20/20...  Training Step: 3958...  Training loss: 1.0558...  2.6327 sec/batch
Epoch: 20/20...  Training Step: 3959...  Training loss: 1.0478...  2.6311 sec/batch
Epoch: 20/20...  Training Step: 3960...  Training loss: 1.0426...  2.5564 sec/batch
```
<font size=4.5>Training Step</font> <font size=3>are the number of batches,</font> <font size=4.5>sec/batch</font> <font size=3>is speed, the</font> <font size=4.5>Training loss</font><font size=3>  is 1.0426 finally.

## Sampling
<br>
<font size=3>
Now that the network is trained, we'll can use it to generate new text. The idea is that we pass in a character, then the network will predict the next character. We can use the new one, to predict the next one. And we keep doing this to generate all new text.
<br></font>
### For example:
```python
checkpoint = tf.train.latest_checkpoint('checkpoints')
samp = sample(checkpoint, 2000, lstm_size, len(vocab), prime="Far")
print(samp)
```
### Episode of generated text ==> 
```
"Yes, I shall see that!"

And all sildey stopped, was futtle and helpless scruminy, and he stopped,
she was terrible, and he was so far from the door, and so sheally careed
for her son stumble that he was not enjoying it to see them in the
sacromman with her son, who could not be sent to all her face;
business of this thoughts at any other in the carriage, that had
sent to him the general work of her, then it will happen of the sound,
though it all of to the serfate sudden, and he went into the doctor, which he
would fing that infered the most intense, thoughts were te get a ballstake,
and he started still, looked at him, stood still of two creature,
which she felt the servant, and at those whise he had left to him that he
would stay over. The prince, and said that they all all done was
things if it had triet, because he child not take another that way.

"That is the hutal prince, because she dod't a came on all the world when
I think of that in the country is applaced to her in the cuttom."

"And that's just at once into the steps. It well time the same way,
talkly, and we've not the sacriment of their position," he said,
truncting himself in the same; "I shall be definite it, and never can he
got on living mestal at alter, and I did my everything--tell me
her husband, that I should go and destroyed."
```
<br>
## Conclusion
<font size=3>
That's the text generated by Input chracters "Far". As we can see almost every word are grammatically correct, the using of punctuation is also correct, the style of the text should be similar to the original novel, only some of the sentences are hard to undestand.<br>
<br><br>

# The Code of the Network:

In [1]:
import time
from collections import namedtuple

import numpy as np
import tensorflow as tf

def get_batches(arr, n_seqs, n_steps):
    '''Create a generator that returns batches of size
       n_seqs x n_steps from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from, shape=(1985223,)
       n_seqs: Batch size, the number of sequences per batch
       n_steps: Number of sequence steps per batch
    '''
    # Get the batch size and number of batches we can make
    batch_size = n_seqs * n_steps # 一个batch的总字符数
    n_batches = len(arr)//batch_size # batch的个数
    
    # Keep only enough characters to make full batches
    arr = arr[:n_batches * batch_size] # 舍弃一些组不成一个batch的字符
    
    # Reshape into n_seqs rows
    arr = arr.reshape((n_seqs, -1)) 
    
    for n in range(0, arr.shape[1], n_steps):
        # The features
        x = arr[:, n:n+n_steps]
        # The targets, shifted by one
        y = np.zeros_like(x)
        y[:, :-1], y[:, -1] = x[:, 1:], x[:, 0]
        yield x, y

In [2]:
def build_inputs(batch_size, num_steps):
    ''' Define placeholders for inputs, targets, and dropout 
    
        Arguments
        ---------
        batch_size: Batch size, number of sequences per batch
        num_steps: Number of sequence steps in a batch
        
    '''
    # Declare placeholders we'll feed into the graph
    inputs = tf.placeholder(tf.int32, shape=(batch_size,num_steps), name='inputs')
    targets = tf.placeholder(tf.int32, shape=(batch_size,num_steps), name='targets')
    
    # Keep probability placeholder for drop out layers
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    
    return inputs, targets, keep_prob

In [3]:
def build_lstm(lstm_size, num_layers, batch_size, keep_prob):
    ''' Build LSTM cell.
    
        Arguments
        ---------
        keep_prob: Scalar tensor (tf.placeholder) for the dropout keep probability 用来给LSTMcell加Dropout
        lstm_size: Size of the hidden layers in the LSTM cells 用来初始化LSTM输出的dimension
        num_layers: Number of LSTM layers 用来堆叠多层LSTM网络
        batch_size: Batch size 用来生成initial_state, 此例中详见下图

    '''
    ### Build the LSTM Cell
    # Use a basic LSTM cell
    lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
    
    # Add dropout to the cell outputs
    drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob = keep_prob)
    
    # Stack up multiple LSTM layers, for deep learning
    cell = tf.contrib.rnn.MultiRNNCell([drop for _ in range(num_layers)])
    
    # batch_size: 一个batch的总字符数
    initial_state = cell.zero_state(batch_size, tf.float32) 
    
    return cell, initial_state

In [4]:
def build_output(lstm_output, in_size, out_size):
    ''' Build a softmax layer, return the softmax output and logits.
    
        Arguments
        ---------
        
        lstm_output: List of output tensors from the LSTM layer
        in_size: Size of the input tensor, for example, size of the LSTM cells
        # LSTM cell的个数, 有几个cell就代表有几层, 至于横向时间展开是为了方便理解, 实际并不存在
        out_size: Size of this softmax layer
        # softmax layer应该和text中出现的字母种类数保持一致, 返回一个概率list, 每个位置代表相应的概率
    '''

    # Reshape output so it's a bunch of rows, one row for each step for each sequence.
    # Concatenate lstm_output over axis 1 (the columns)
    seq_output = tf.concat(lstm_output, axis=1)
    # Reshape seq_output to a 2D tensor with lstm_size columns
    # reshape中的-1表示推断, 即给出其中一个维度参数的情况下, 推断出另一个维度应该是多少
    # in_size表示LSTM cell的个数, 这一步的输出应该是每层cell的输出列成一列
    x = tf.reshape(seq_output, [-1,in_size]) 
    
    # Connect the RNN outputs to a softmax layer
    with tf.variable_scope('softmax'):
        # Create the weight and bias variables here
        softmax_w = tf.Variable(tf.truncated_normal([in_size,out_size], stddev=0.1, dtype=tf.float32))
        softmax_b = tf.Variable(tf.zeros(out_size))
    
    # Since output is a bunch of rows of RNN cell outputs, logits will be a bunch
    # of rows of logit outputs, one for each step and sequence
    logits = tf.matmul(x, softmax_w) + softmax_b
    
    # Use softmax to get the probabilities for predicted characters
    out = tf.nn.softmax(logits, name='prediction')
    
    return out, logits

In [6]:
def build_loss(logits, targets, lstm_size, num_classes):
    ''' Calculate the loss from the logits and the targets.
    
        Arguments
        ---------
        logits: Logits from final fully connected layer
        targets: Targets for supervised learning
        lstm_size: Number of LSTM hidden units
        num_classes: Number of classes in targets
        
    '''
    
    # One-hot encode targets and reshape to match logits, one row per sequence per step
    y_one_hot = tf.one_hot(targets, num_classes)
    y_reshaped = tf.reshape(y_one_hot, shape=(logits.shape))
    
    # Softmax cross entropy loss
    loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_reshaped)
    loss = tf.reduce_mean(loss)
    
    return loss

In [7]:
def build_optimizer(loss, learning_rate, grad_clip):
    ''' Build optmizer for training, using gradient clipping.
    
        Arguments:
        loss: Network loss
        learning_rate: Learning rate for optimizer
    
    '''
    
    # Optimizer for training, using gradient clipping to control exploding gradients
    
    # 只要Variable变量的 trainable = True(默认为True), 则tf.trainable_variables()会收集所有Variable类的实例(变量).
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), grad_clip)
    train_op = tf.train.AdamOptimizer(learning_rate)
    optimizer = train_op.apply_gradients(zip(grads, tvars))
    
    return optimizer

In [8]:
class CharRNN:
    
    def __init__(self, num_classes, batch_size=64, num_steps=50, 
                       lstm_size=128, num_layers=2, learning_rate=0.001, 
                       grad_clip=5, sampling=False):
    
        # When we're using this network for sampling later, we'll be passing in
        # one character at a time, so providing an option for that
        if sampling == True:
            batch_size, num_steps = 1, 1
        else:
            batch_size, num_steps = batch_size, num_steps
            
        # Reset graph    
        tf.reset_default_graph()
        
        #build input placeholder
        self.inputs, self.targets, self.keep_prob = build_inputs(batch_size, num_steps) 
        
        # Build the LSTM cell
        self.lstm, self.initial_state = build_lstm(lstm_size, num_layers, batch_size, self.keep_prob)
        
        ### Run the data through the RNN layer
        # one-hot encode the input token
        x_one_hot = tf.one_hot(self.inputs, num_classes)
        
        # Run each sequence step through RNN with tf.nn.dynamic_run
        outputs, state = tf.nn.dynamic_rnn(self.lstm, x_one_hot, initial_state=self.initial_state)
        self.final_state = state
        
        # Get softmax prediction and logits
        self.prediction, self.logits = build_output(outputs, lstm_size, num_classes)
        
        # Loss and optimizer (with gradient clipping)
        self.loss =  build_loss(self.logits, self.targets, lstm_size, num_classes)
        self.optimizer = build_optimizer(self.loss, learning_rate, grad_clip)

In [ ]:
###### Train the network ######
epochs = 20
# Save every N iterations
save_every_n = 200

model = CharRNN(len(vocab), batch_size=batch_size, num_steps=num_steps,
                lstm_size=lstm_size, num_layers=num_layers, 
                learning_rate=learning_rate)

saver = tf.train.Saver(max_to_keep=100)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # Use the line below to load a checkpoint and resume training
#     saver.restore(sess, 'checkpoints/i78200_l128.ckpt')
    counter = 0
    for e in range(epochs):
        # Train network
        new_state = sess.run(model.initial_state)
        loss = 0
        for x, y in get_batches(encoded, batch_size, num_steps):
            counter += 1
            start = time.time()
            feed = {model.inputs: x,
                    model.targets: y,
                    model.keep_prob: keep_prob,
                    model.initial_state: new_state}
            batch_loss, new_state, _ = sess.run([model.loss, 
                                                 model.final_state, 
                                                 model.optimizer], 
                                                 feed_dict=feed)
            
            end = time.time()
            print('Epoch: {}/{}... '.format(e+1, epochs),
                  'Training Step: {}... '.format(counter),
                  'Training loss: {:.4f}... '.format(batch_loss),
                  '{:.4f} sec/batch'.format((end-start)))
        
            if (counter % save_every_n == 0):
                saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))
    
    saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))